In [1]:
# TODO: SSD model (PriorBox)
import keras.backend as K
from keras.engine.topology import InputSpec
from keras.engine.topology import Layer
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


## Series of boxes [xmin, ymin, xmax, ymax] and variances
This layer is just providing each layers [xmin, ymin, xmax, ymax] which units maps from layer to image.<br/>
Because training data is all about with real image size.
- Note
> `I wonder that in this layer, tensor will be trained like Neuron.`

In [2]:
class PriorBox(Layer):
    
    def __init__(self, img_size, min_size, max_size=None, aspect_ratios=None,
                flip=True, variances=[0.1], clip=True, **kwargs):
        self.waxis = 2
        self.haxis = 1
        self.img_size = img_size
        self.min_size = min_size
        self.max_size = max_size
        self.aspect_ratios = [1.0]
        if max_size:
            self.aspect_ratios.append(1.0)
        if aspect_ratios:
            for ar in aspect_ratios:
                if ar in self.aspect_ratios:
                    continue
                self.aspect_ratios.append(ar)
                if flip:
                    self.aspect_ratios.append(1.0 / ar)
        self.variances = np.array(variances)
        self.clip = True
        super(PriorBox, self).__init__(**kwargs)
        
    def compute_output_shape(self, input_shape):
        num_priors_ = len(self.aspect_ratios)
        layer_width = input_shape[self.waxis]
        layer_height = input_shape[self.haxis]
        num_boxes = num_priors_ * layer_width * layer_height
        # 8 means shape of (xmin, ymin, xmax, ymax, variances{4})
        return (input_shape[0], num_boxes, 8)
    
    def call(self, x, mask=None):
        if hasattr(x, '_keras_shape'):
            input_shape = x._keras_shape
        elif hasattr(K, 'int_shape'):
            input_shape = K.int_shape(x)
            
        layer_width = input_shape[self.waxis]
        layer_height = input_shape[self.haxis]
        
        img_width, img_height = img_size[:2]
        
        box_widths = []
        box_heights = []
        
        for ar in self.aspect_ratios:
            if ar == 1 and len(box_widths) == 0:
                box_widths.append(self.min_size)
                box_heights.append(self.min_size)
            elif ar == 1 and len(box_widths) > 0:
                # sqrt(min*max)
                box_widths.append(np.sqrt(self.min_size * self.max_size))
                box_heights.append(np.sqrt(self.min_size * self.max_size))
            elif ar != 1:
                # ratio of width:height
                box_widths.append(self.min_size * np.sqrt(ar))
                box_heights.append(self.min_size / np.sqrt(ar))
        
        # half length, for calculate step length (image_size)
        box_widths = 0.5 * np.array(box_widths)
        box_heights = 0.5 * np.array(box_heights)
        
        # define centers of prior boxes
        # mapping steps from layer to image
        step_x = img_width / layer_width
        step_y = img_height / layer_height
        
        # list of box center positions (x, y)
        # note here, it's restricted by image_size 
        linx = np.linspace(0.5 * step_x, img_width - 0.5 * step_x, layer_width)
        liny = np.linspace(0.5 * step_y, img_height - 0.5 * step_y, layer_height)
        
        centers_x, centers_y = np.meshgrid(linx, liny)
        centers_x = centers_x.reshape(-1, 1)
        centers_y = centers_y.reshape(-1, 1)
        
        # define xmin, ymin, xmax, ymax of priors boxes
        num_priors_ = len(self.aspect_ratios)
        # (x, y) of center box
        prior_boxes = np.concatenate((centers_x, centers_y), axis=1)
        prior_boxes = np.tile(prior_boxes, (1, 2 * num_priors_))
        # [xmin, ymin, xmax, ymax]
        prior_boxes[:, ::4] -= box_widths
        prior_boxes[:, 1::4] -= box_heights
        prior_boxes[:, 2::4] += box_widths
        prior_boxes[:, 3::3] += box_heights
        # regularization
        prior_boxes[:, ::2] /= img_width
        prior_boxes[:, 1::2] /= img_height
        prior_boxes = prior_boxes.reshape(-1, 4)
        
        if self.clip:
            prior_boxes = np.minimum(np.maximum(prior_boxes, 0.0), 1.0)
        if len(self.variances) == 1:
            variances = np.ones((num_boxes, 4)) * self.variances[0]
        elif len(self.variances) == 4:
            variances = np.tile(self.variances, (num_boxes, 1))
            
        prior_boxes = np.concatenate((prior_boxes, variances), axis=1)
        prior_boxes_tensor = K.expand_dims(K.variable(prior_boxes), 0)
        pattern = [tf.shape(x)[0], 1, 1]
        prior_boxes_tensor = tf.tile(prior_boxes_tensor, pattern)
        
        return prior_boxes_tensor